Wikipedia

In [4]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [5]:
from langchain.utilities import WikipediaAPIWrapper

In [6]:
wikipedia = WikipediaAPIWrapper()

In [7]:
wikipedia.run("Large Language Models")

'Page: Large language model\nSummary: A large language model (LLM) is a language model notable for its ability to achieve general-purpose language generation and understanding. LLMs acquire these abilities by learning statistical relationships from text documents during a computationally intensive self-supervised and semi-supervised training process. LLMs are artificial neural networks, the largest and most capable of which are built with a transformer-based architecture. Some recent implementations are based on other architectures, such as recurrent neural network variants and Mamba (a state space model).LLMs can be used for text generation, a form of generative AI, by taking an input text and repeatedly predicting the next token or word. Up to 2020, fine tuning was the only way a model could be adapted to be able to accomplish specific tasks. Larger sized models, such as GPT-3, however, can be prompt-engineered to achieve similar results. They are thought to acquire knowledge about s

Python REPL

In [8]:
from langchain.utilities import PythonREPL

In [9]:
python_repl = PythonREPL()

In [10]:
python_repl.run("print('Hello Rajan')")

Python REPL can execute arbitrary code. Use with caution.


'Hello Rajan\n'

Duck Duck Go

In [11]:
from langchain.tools import DuckDuckGoSearchRun

In [12]:
search = DuckDuckGoSearchRun()

In [13]:
search.run("Bombay Stock Exchange")

"Get the latest live index levels and daily trend for the major indices on BSE and NSE along with their advance-decline numbers. Search, select and invest in stocks that fit your investment criteria, scan stocks by sector, industry, PSU/non-PSU and more. The BSE is Asia's first exchange and lists over 5,300 companies in various sectors. It enables investors to trade equities, currencies, debt instruments, derivatives, and mutual funds. It is one of the largest and most active exchanges in the world, along with NYSE, Nasdaq, London, and Shanghai. Bombay Stock Exchange, S&P BSE Indices, Sensex - What is it? Established in 1875 in Dalal Street of Mumbai, BSE Limited (formerly, Bombay Stock Exchange) is the oldest stock exchange in Asia. Sensex, otherwise known as the S&P BSE Sensex index, is the benchmark index of the Bombay Stock Exchange (BSE). It is composed of 30 of the largest and most actively-traded stocks on the BSE ... India's booming stock market. Investors are buying into a gro

Combination of these Agents

In [14]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.5)

In [15]:
from langchain.agents import Tool

tools = [
    Tool(
        name = "python repl",
        func=python_repl.run,
        description="useful for when you need to use python to answer a question. You should input python code"
    )
]

C:\Users\rajan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [16]:
wikipedia_tool = Tool(
    name='wikipedia',
    func= wikipedia.run,
    description="Useful for when you need to look up a topic, country or person on wikipedia"
)

duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func= search.run,
    description="Useful for when you need to do a search on the internet to find information that another tool can't find. Be specific with your input."
)

In [17]:
tools.append(wikipedia_tool)
tools.append(duckduckgo_tool)
print(tools)

[Tool(name='python repl', description='useful for when you need to use python to answer a question. You should input python code', func=<bound method PythonREPL.run of PythonREPL(globals={'__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provides direct access to all 'built-in'\nidentifiers of Python; for example, builtins.len is\nthe full name for the built-in function len().\n\nThis module is not normally accessed explicitly by most\napplications, but can be useful in modules that provide\nobjects with the same name as a built-in value, but in\nwhich the built-in of that name is also needed.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'), '__build_class__': <built-in function __build_class__>, '__import__': <built-in function __import__>, 'abs': <built-in function abs

In [28]:
from langchain.agents import initialize_agent, AgentType

zero_shot_agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=1,
    max_token_limit=256
)

In [19]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

python repl: useful for when you need to use python to answer a question. You should input python code
wikipedia: Useful for when you need to look up a topic, country or person on wikipedia
DuckDuckGo Search: Useful for when you need to do a search on the internet to find information that another tool can't find. Be specific with your input.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python repl, wikipedia, DuckDuckGo Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [20]:
zero_shot_agent.invoke("What is the age of Joe Biden?")



> Entering new AgentExecutor chain...


 I should use Wikipedia to look up Joe Biden's page
Action: Wikipedia
Action Input: "Joe Biden"
Observation: Wikipedia is not a valid tool, try one of [python repl, wikipedia, DuckDuckGo Search].
Thought:

> Finished chain.


{'input': 'What is the age of Joe Biden?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [21]:
zero_shot_agent.run('what is the current price of bitcoin?')



> Entering new AgentExecutor chain...


C:\Users\rajan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 I should use DuckDuckGo Search to find the current price of bitcoin.
Action: DuckDuckGo Search
Action Input: "current price of bitcoin"
Observation: The current price of Bitcoin is $41,673.63 per BTC, according to Coinbase. You can use the BTC to USD calculator to convert any amount of BTC to USD or vice versa. See the market cap, volume, circulating supply and historical data of Bitcoin and other crypto assets. Binance provides live price of Bitcoin (BTC) in USD, with a current market cap of $ 779.40B USD and a 24-hour trading volume of $ 29.27B USD. Learn about Bitcoin's history, volatility, supply, and future halving event. Get the latest Bitcoin price, live chart, and news on Blockchain.com, the world's most popular crypto wallet. Learn about Bitcoin's history, mining, supply, and features, and compare it with other cryptocurrencies. Find the current price of Bitcoin (BTC) in USD and other currencies, as well as the latest news and market data on the leading cryptocurrency. Compar

'Agent stopped due to iteration limit or time limit.'

C:\Users\rajan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [22]:
zero_shot_agent.run('Write a function to check if 11 a prime number and test it')



> Entering new AgentExecutor chain...
 I should first define the function and then test it with different inputs
Action: python repl
Action Input: 
def is_prime(n):
    if n == 1:
        return False
    for d in range(2, n):
        if n % d == 0:
            return False
    return True

is_prime(11)
Observation: 
Thought:

> Finished chain.


'Agent stopped due to iteration limit or time limit.'